## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [1]:
!which jupyter

/home/yz6/.virtualenvs/gan_env/bin/jupyter


In [2]:


import os.path as osp

from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph

/home/yz6/.virtualenvs/gan_env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [11]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()

Give me the class name (e.g. "chair"): airplane


Load Point-Clouds

In [12]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

4045 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed here). For more details please use print, etc.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of this many epochs)
    'saver_step': 10       (# how many epochs to save neural-network)

In [13]:
train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [14]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
conf.save(osp.join(train_dir, 'configuration'))

Build AE Model.

In [15]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("single_class_ae_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

Tensor("single_class_ae_2/Max:0", shape=(?, 128), dtype=float32)
Building

Train the AE (save output to train_stats.txt) 

In [ ]:
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.5557', 'loss=', '0.077831632')
INFO:tensorflow:../data/single_class_ae/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.5534', 'loss=', '0.051425679')
('Epoch:', '0003', 'training time (minutes)=', '0.5535', 'loss=', '0.048744040')
('Epoch:', '0004', 'training time (minutes)=', '0.5532', 'loss=', '0.046022463')
('Epoch:', '0005', 'training time (minutes)=', '0.5532', 'loss=', '0.044571394')
('Epoch:', '0006', 'training time (minutes)=', '0.5533', 'loss=', '0.043440614')
('Epoch:', '0007', 'training time (minutes)=', '0.5531', 'loss=', '0.041913794')
('Epoch:', '0008', 'training time (minutes)=', '0.5530', 'loss=', '0.041085395')
('Epoch:', '0009', 'training time (minutes)=', '0.5531', 'loss=', '0.040669892')
('Epoch:', '0010', 'training time (minutes)=', '0.5531', 'loss=', '0.039997117')
INFO:tensorflow:../data/single_class_ae/models.ckpt-10 is not in all_model_che

('Epoch:', '0090', 'training time (minutes)=', '0.5530', 'loss=', '0.031549776')
INFO:tensorflow:../data/single_class_ae/models.ckpt-90 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0091', 'training time (minutes)=', '0.5532', 'loss=', '0.030658772')
('Epoch:', '0092', 'training time (minutes)=', '0.5530', 'loss=', '0.031326662')
('Epoch:', '0093', 'training time (minutes)=', '0.5531', 'loss=', '0.031053642')
('Epoch:', '0094', 'training time (minutes)=', '0.5531', 'loss=', '0.030742591')
('Epoch:', '0095', 'training time (minutes)=', '0.5530', 'loss=', '0.031167185')
('Epoch:', '0096', 'training time (minutes)=', '0.5531', 'loss=', '0.030951000')
('Epoch:', '0097', 'training time (minutes)=', '0.5532', 'loss=', '0.030654038')
('Epoch:', '0098', 'training time (minutes)=', '0.5532', 'loss=', '0.030747567')
('Epoch:', '0099', 'training time (minutes)=', '0.5532', 'loss=', '0.030696890')
('Epoch:', '0100', 'training time (minutes)=', '0.5534', 'loss=', '0.03051063

('Epoch:', '0179', 'training time (minutes)=', '0.5532', 'loss=', '0.029004787')
('Epoch:', '0180', 'training time (minutes)=', '0.5530', 'loss=', '0.028608417')
INFO:tensorflow:../data/single_class_ae/models.ckpt-180 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0181', 'training time (minutes)=', '0.5532', 'loss=', '0.029269147')
('Epoch:', '0182', 'training time (minutes)=', '0.5532', 'loss=', '0.028595845')
('Epoch:', '0183', 'training time (minutes)=', '0.5531', 'loss=', '0.028461538')
('Epoch:', '0184', 'training time (minutes)=', '0.5529', 'loss=', '0.028810056')


Get some reconstuctions and latent-codes.

In [ ]:
feed_pc, feed_model_names, _ = all_pc_data.next_batch(10)
reconstructions = ae.reconstruct(feed_pc)
latent_codes = ae.transform(feed_pc)